In [ ]:
! pip install git+https://github.com/openai/whisper.git -q
! pip install pydub

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import whisper
import os
import pydub
import nltk
import shutil
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
shutil.rmtree('/content/openai-whisper-webapp/audio-chunks', ignore_errors=True)

In [ ]:
model = whisper.load_model("medium")

100%|█████████████████████████████████████| 1.42G/1.42G [00:15<00:00, 96.3MiB/s]


In [ ]:
from pydub.silence import split_on_silence

In [ ]:
model.device

device(type='cuda', index=0)

In [ ]:
!git clone https://github.com/petewarden/openai-whisper-webapp
from IPython.display import Audio

Cloning into 'openai-whisper-webapp'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 29 (delta 11), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (29/29), 580.75 KiB | 6.05 MiB/s, done.


In [ ]:
src="gopro.mp3"

In [ ]:
def transcribe(audio):

    audio = whisper.load_audio(audio)
    audio = whisper.pad_or_trim(audio)

    mel = whisper.log_mel_spectrogram(audio).to(model.device)

    # _, probs = model.detect_language(mel)
    # print(f"Detected language: {max(probs, key=probs.get)}")

    options = whisper.DecodingOptions()
    result = whisper.decode(model, mel, options)
    return result.text

In [ ]:
transcribe(src)

"GoPro, they make action cameras, drones that one time, and terrible investments. Now GoPro wasn't the first company to ever make a compact and durable camera and according to many, they're not even the best at it. But what GoPro did do is help to popularize the entire industry and fuel a competition-based arms race within the durable small action camera space."

In [ ]:
dst="audio.wav"
sound = pydub.AudioSegment.from_mp3(src)
sound.export(dst, format="wav")

<_io.BufferedRandom name='audio.wav'>

In [ ]:
def large_transcription(path):
    sound = pydub.AudioSegment.from_wav(path)
    chunks = pydub.silence.split_on_silence(sound,
        min_silence_len = 300,
        silence_thresh = sound.dBFS-14,
        keep_silence=50,
    )
    whole_text = ""
    cf = "openai-whisper-webapp/audio-chunks"
    if not os.path.isdir(cf):
      os.mkdir(cf)

    for i, audio_chunk in enumerate(chunks, start=1):
        chunk_filename = os.path.join(cf, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")

    for i, audio_chunk in enumerate(chunks, start=1):
        whole_text=whole_text+" "+transcribe("/content/openai-whisper-webapp/audio-chunks"+f"/chunk{i}.wav")
    # for i, audio_chunk in enumerate(chunks, start=1):
    #     print(transcribe("/content/openai-whisper-webapp/audio-chunks"+f"/chunk{i}.wav"), '\n')    
    return whole_text

In [ ]:
final=large_transcription(dst)
final

" GoPro, they make action cameras, drones that one time, and terrible investments. Now GoPro wasn't the first company to ever make a compact and durable camera, and according to many, they're not even the best at it. But what GoPro did do is help to popularize the entire industry and fuel a competition-based arms race. thin. the Durable. small action camera space. And as somebody that loves technology and improvements, I can appreciate the contribution. GoPro made to the second. However... GoPro does have a known Pretty cool. flaw with their products even now on their 11th generation. And that's that they simply don't always work. I am over GoPro. I'm officially over GoPro. This company, man. I don't know what's going on. GoPro just overheated after like a minute and a half of being on. Uh oh. Hero 11. So Hero 11 just went to... after. first. The first video I tried to take on the GoPro 11 Black I got the screen of death. constantly loose footage. I just bought this camera a week ago. 

In [ ]:
def _create_frequency_table(text_string) -> dict:

    stopWords = set(stopwords.words("english"))
    words = word_tokenize(text_string)
    ps = PorterStemmer()

    freqTable = dict()
    for word in words:
        word = ps.stem(word)
        if word in stopWords:
            continue
        if word in freqTable:
            freqTable[word] += 1
        else:
            freqTable[word] = 1

    return freqTable

In [ ]:
def _score_sentences(sentences, freqTable) -> dict:
    sentenceValue = dict()

    for sentence in sentences:
        word_count_in_sentence = (len(word_tokenize(sentence)))
        for wordValue in freqTable:
            if wordValue in sentence.lower():
                if sentence[:10] in sentenceValue:
                    sentenceValue[sentence[:10]] += freqTable[wordValue]
                else:
                    sentenceValue[sentence[:10]] = freqTable[wordValue]

        sentenceValue[sentence[:10]] = sentenceValue[sentence[:10]] // word_count_in_sentence

    return sentenceValue

In [ ]:
def _find_average_score(sentenceValue) -> int:
    sumValues = 0
    for entry in sentenceValue:
        sumValues += sentenceValue[entry]
        
    average = int(sumValues / len(sentenceValue))

    return average

In [ ]:
def _generate_summary(sentences, sentenceValue, threshold):
    sentence_count = 0
    summary = ''

    for sentence in sentences:
        if sentence[:10] in sentenceValue and sentenceValue[sentence[:10]] > (threshold):
            summary += " " + sentence
            sentence_count += 1

    return summary

In [ ]:
test='''
World War II or the Second World War, often abbreviated as WWII or WW2, was a global conflict that lasted from 1939 to 1945. The vast majority of the world's countries, including all of the great powers, fought as part of two opposing military alliances: the Allies (lead by the Soviet Union, United Kingdom, United States and China) and the Axis (lead by Germany, Japan, and Italy). Many participants threw their economic, industrial, and scientific capabilities behind this total war, blurring the distinction between civilian and military resources. Aircraft played a major role, enabling the strategic bombing of population centres and the only two nuclear weapons ever used in war.

World War II was by far the deadliest conflict in human history; it resulted in 70 to 85 million fatalities, mostly among civilians. Tens of millions died due to genocides (including the Holocaust), starvation, massacres, and disease. In the wake of the Axis defeat, Germany and Japan were occupied, and war crimes tribunals were conducted against German and Japanese leaders.

The causes of World War II are debated, but contributing factors included the Second Italo-Ethiopian War, Spanish Civil War, Second Sino-Japanese War, Soviet–Japanese border conflicts, rise of fascism in Europe and rising European tensions since World War I. World War II is generally considered to have begun on 1 September 1939, when Nazi Germany, under Adolf Hitler, invaded Poland. The United Kingdom and France subsequently declared war on Germany on 3 September. Under the Molotov–Ribbentrop Pact of August 1939, Germany and the Soviet Union had partitioned Poland and marked out their "spheres of influence" across Finland, Estonia, Latvia, Lithuania and Romania. From late 1939 to early 1941, in a series of campaigns and treaties, Germany conquered or controlled much of continental Europe, and formed the Axis alliance with Italy and Japan (with other countries later). Following the onset of campaigns in North Africa and East Africa, and the fall of France in mid-1940, the war continued primarily between the European Axis powers and the British Empire, with war in the Balkans, the aerial Battle of Britain, the Blitz of the United Kingdom, and the Battle of the Atlantic. On 22 June 1941, Germany led the European Axis powers in an invasion of the Soviet Union, opening the Eastern Front, the largest land theatre of war in history.

Japan, which aimed to dominate Asia and the Pacific, was at war with the Republic of China by 1937. In December 1941, Japan attacked American and British territories with near-simultaneous offensives against Southeast Asia and the Central Pacific, including an attack on the US fleet at Pearl Harbor which resulted in the United States declaring war against Japan. The European Axis powers declared war on the United States in solidarity. Japan soon captured much of the western Pacific, but its advances were halted in 1942 after losing the critical Battle of Midway; later, Germany and Italy were defeated in North Africa and at Stalingrad in the Soviet Union. Key setbacks in 1943—including a series of German defeats on the Eastern Front, the Allied invasions of Sicily and the Italian mainland, and Allied offensives in the Pacific—cost the Axis powers their initiative and forced them into strategic retreat on all fronts. In 1944, the Western Allies invaded German-occupied France, while the Soviet Union regained its territorial losses and turned towards Germany and its allies. During 1944 and 1945, Japan suffered reversals in mainland Asia, while the Allies crippled the Japanese Navy and captured key western Pacific islands.

The war in Europe concluded with the liberation of German-occupied territories and the invasion of Germany by the Western Allies and the Soviet Union, culminating in the Fall of Berlin to Soviet troops, Hitler's suicide, and the German unconditional surrender on 8 May 1945. Following the refusal of Japan to surrender on the terms of the Potsdam Declaration (issued 26 July 1945), the United States dropped the first atomic bombs on the Japanese cities of Hiroshima on 6 August and Nagasaki on 9 August. Faced with an imminent invasion of the Japanese archipelago, the possibility of additional atomic bombings, and the Soviet Union's declared entry into the war against Japan on the eve of invading Manchuria, Japan announced on 10 August its intention to surrender, signing a surrender document on 2 September 1945.

World War II changed the political alignment and social structure of the globe. The United Nations was established to foster international co-operation and prevent future conflicts,[1] with the victorious great powers—China, France, the Soviet Union, the United Kingdom, and the United States—becoming the permanent members of its Security Council. The Soviet Union and the United States emerged as rival superpowers, setting the stage for the nearly half-century-long Cold War. In the wake of European devastation, the influence of its great powers waned, triggering the decolonisation of Africa and Asia. Most countries whose industries had been damaged moved towards economic recovery and expansion. Political and economic integration, especially in Europe, began as an effort to forestall future hostilities, end pre-war enmities and forge a sense of common identity.
'''


In [ ]:
freq_table = _create_frequency_table(test)

sentences = sent_tokenize(test)

sentence_scores = _score_sentences(sentences, freq_table)

threshold = _find_average_score(sentence_scores)

summary = _generate_summary(sentences, sentence_scores,1.1* threshold)

print(summary)

 
World War II or the Second World War, often abbreviated as WWII or WW2, was a global conflict that lasted from 1939 to 1945. Aircraft played a major role, enabling the strategic bombing of population centres and the only two nuclear weapons ever used in war. World War II was by far the deadliest conflict in human history; it resulted in 70 to 85 million fatalities, mostly among civilians. Tens of millions died due to genocides (including the Holocaust), starvation, massacres, and disease. In the wake of the Axis defeat, Germany and Japan were occupied, and war crimes tribunals were conducted against German and Japanese leaders. World War II is generally considered to have begun on 1 September 1939, when Nazi Germany, under Adolf Hitler, invaded Poland. On 22 June 1941, Germany led the European Axis powers in an invasion of the Soviet Union, opening the Eastern Front, the largest land theatre of war in history. Japan, which aimed to dominate Asia and the Pacific, was at war with the R